In [134]:
import sys
import requests
from bs4 import BeautifulSoup
import unicodedata
import itertools
import itertools
import pprint
import re

URL = "https://arm-software.github.io/acle/neon_intrinsics/advsimd.html"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
intrinsics = [unicodedata.normalize('NFKD', code.text) for code in soup.select("tbody > tr > td:nth-child(1) > code")]
archs =  [unicodedata.normalize('NFKD', code.text) for code in soup.select("tbody > tr > td:nth-child(5) > code")]

decls = [' '.join(entry.split()).replace('( ', '(') for entry in intrinsics]

decl_archs = zip(decls, archs)


In [135]:
def grouper(iterable, n, *, incomplete='fill', fillvalue=None):
    "Collect data into non-overlapping fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, fillvalue='x') --> ABC DEF Gxx
    # grouper('ABCDEFG', 3, incomplete='strict') --> ABC DEF ValueError
    # grouper('ABCDEFG', 3, incomplete='ignore') --> ABC DEF
    args = [iter(iterable)] * n
    if incomplete == 'fill':
        return itertools.zip_longest(*args, fillvalue=fillvalue)
    if incomplete == 'strict':
        return zip(*args, strict=True)
    if incomplete == 'ignore':
        return zip(*args)
    else:
        raise ValueError('Expected fill, strict, or ignore')

In [136]:
lengths = ["8", "16", "32", "64", "128", "256"]

def prefix_all(prefix, list):
  return [prefix + elem for elem in list]

types = {
  *prefix_all('u', lengths), # unsigned integers
  *prefix_all('s', lengths), # signed integers
  *prefix_all('f', lengths), # floating point
  *prefix_all('p', lengths), # polynomial
  'bf16', # brain floating point
}


class NEONIdent:
  suffixes = ['q', # Q-register
              'w', # Widening, i.e one of the inputs is longer than the other and the output will be placed in a longer type
              'l'] # Long, i.e. the output type is twice as long as the inputs

  def __init__(self, func_name):
    parts = func_name.lstrip('_').split('_')

    if parts[0][0] == 'v':
      parts[0] = parts[0][1:] # remove first char

    if parts[0][-1] in NEONIdent.suffixes:
      self.suffix = parts[0][-1]
      if self.suffix == 'q':
        parts[0] = parts[0][:-1]
    else:
      self.suffix = None
      
    
    name_types = [type for type in parts if type in types]
    parts = [part for part in parts if part not in types]
    
    if len(name_types) == 1:
      [out_type, in_type] =  name_types * 2
    elif len(name_types) == 2:
      [out_type, in_type] = name_types
    else:
      [out_type, in_type] = [None, None]
    
    self.types = {'out' : out_type, 'in': in_type}

    if 'n' in parts:
      parts.remove('n')
      self.const = 'n'
    elif 'lane' in parts:
      parts.remove('lane')
      self.const = 'lane'
    elif 'laneq' in parts:
      parts.remove('laneq')
      self.const = 'lane'
    else:
      self.const = None
    
    self.name = '_'.join(parts)


  def __repr__(self):
    return f'NEON:{{ name: "{self.name}", suffix: {self.suffix}, types: {self.types}}}'

In [137]:
name_map = {
  "add": "add",
  "addl": "add_long",
  "addw": "add",
  "hadd": "add_halve",
  "rhadd": "add_halve_round",
  "addhn": "add_narrow_high",
  "raddhn": "add_round_narrow_high",
  "qadd": "add_saturate",
  "mul": "multiply",
  "mla": "multiply_add",
  "mls": "multiply_subtract",
  "mlal": "multiply_add_long",
  "mlsl": "multiply_subtract_long",
  "fma": "multiply_add_fused",
  "fms": "multiply_subtract_fused",
  "qdmulh": "multiply_double_saturate_high",
  "qrdmulh": "multiply_double_round_saturate_high",
  "qdmull": "multiply_double_saturate_long",
  "qdmlal": "multiply_double_add_saturate_long",
  "qdmlsl": "multiply_double_subtract_saturate_long",
  "mull": "multiply_long",
  "sub": "subtract",
  "subl": "subtract_long",
  "subw": "subtract",
  "hsub": "subtract_high",
  "subhn": "subtract_narrow_high",
  "rsubhn": "subtract_round_narrow_high",
  "qsub": "subtract_saturate",
  "abd": "subtract_abs",
  "abdl": "subtract_abs_long",
  "aba": "subtract_abs_add",
  "abal": "subtract_abs_add",
  "abs": "abs",
  "qabs": "abs_saturate",
  "max": "max",
  "min": "min",
  "recpe": "reciprocal_estimate",
  "recps": "reciprocal_step",
  "rsqrte": "reciprocal_sqrt_estimate",
  "rsqrts": "reciprocal_sqrt_step",
  "padd": "add_pairwise",
  "paddl": "add_pairwise_long",
  "padal": "add_pairwise_accumulate_long",
  "pmax": "max_pairwise",
  "pmin": "min_pairwise",
  "ce": "equal",
  "ceq": "equal",
  "cge": "greater_than_or_equal",
  "cle": "less_than_or_equal",
  "cgt": "greater_than",
  "clt": "less_than",
  "cage": "absolute_greater_than_or_equal",
  "cale": "absolute_less_than_or_equal",
  "cagt": "absolute_greater_than",
  "calt": "absolute_less_than",
  "tst": "and_test_nonzero",
  "shl": "shift_left",
  "qshl": "shift_left_saturate",
  "qshlu": "shift_left_unsigned_saturate",
  "rshl": "shift_left_round",
  "qrshl": "shift_left_round_saturate",
  "shll": "shift_left_long",
  "sli": "shift_left_insert",
  "shr": "shift_right",
  "rshr": "shift_right_round",
  "sra": "shift_right_accumulate",
  "rsra": "shift_right_accumulate_round",
  "shrn": "shift_right_narrow",
  "qshrun": "qshrun",
  "qshrn": "qshrn",
  "qrshrun": "shift_right_unsigned_saturate_narrow",
  "qrshrn": "shift_right_saturate_narrow",
  "rshrn": "shift_right_round_saturate_narrow",
  "sri": "shift_right_insert",
  "cvt": "convert",
  "reinterpret": "reinterpret",
  "movn": "move_narrow",
  "movn_high": "move_high_narrow",
  "movl": "move_long",
  "qmovn": "move_saturate_narrow",
  "qmovun": "move_unsigned_saturate_narrow",
  "neg": "negate",
  "qneg": "negate_saturate",
  "mvn": "not_bitwise",
  "and": "and_bitwise",
  "orr": "or_bitwise",
  "eor": "xor_bitwise",
  "orn": "or_not_bitwise",
  "cls": "count_leading_sign_bits",
  "clz": "count_leading_zero_bits",
  "cnt": "count_active_bits",
  "bic": "clear_bitwise",
  "bsl": "select_bitwise",
  "create": "create",
  "dup": "duplicate_element",
  "mov": "move",
  "combine": "combine",
  "get_high": "get_high",
  "get_low": "get_low",
  "get": "get",
  "ext": "extract",
  "rev64": "reverse_64bit",
  "rev32": "reverse_32bit",
  "rev16": "reverse_16bit",
  "zip": "zip",
  "uzp": "unzip",
  "trn": "transpose",
  "set": "set",
  "ld1": "load1",
  "ld1_dup": "load1_duplicate",
  "ld2": "load2",
  "ld3": "load3",
  "ld4": "load4",
  "ld2_dup": "load2_duplicate",
  "ld3_dup": "load3_duplicate",
  "ld4_dup": "load4_duplicate",
  "ld1_x2": "load1_x2",
  "ld1_x3": "load1_x3",
  "ld1_x4": "load1_x4",
  "st1": "store1",
  "st2": "store2",
  "st3": "store3",
  "st4": "store4",
  "st1_x2": "store1_x2",
  "st1_x3": "store1_x3",
  "st1_x4": "store1_x4",
  "tbl1": "table_lookup1",
  "tbx1": "table_lookup_extension1",
  "tbl2": "table_lookup2",
  "tbl3": "table_lookup3",
  "tbl4": "table_lookup4",
  "tbx2": "table_lookup_extension2",
  "tbx3": "table_lookup_extension3",
  "tbx4": "table_lookup_extension4"
}

In [138]:
import functools


type_order = [
  "uint8x8_t",
  "uint8x16_t",
  "int8x8_t",
  "int8x16_t",
  "uint16x4_t",
  "uint16x8_t",
  "int16x4_t",
  "int16x8_t",
  "uint32x2_t",
  "uint32x4_t",
  "int32x2_t",
  "int32x4_t",
  "uint64x1_t",
  "uint64x2_t",
  "uint32x2_t",
  "uint32x4_t",
  "float16x4_t",
  "float16x8_t",
  "float32x2_t",
  "float32x4_t",
  "poly8x8_t",
  "poly16x4_t",
]

class Var:
  type_map = { key:value for (value,key) in enumerate(type_order) }
  
  def __init__(self, string):
    components = string.split()
    self.ident = components.pop(-1)
    self.type = ' '.join(components)

  def __str__(self):
    return f"{self.type} {self.ident}"

  def __repr__(self):
    return f'Var:{{ type: "{self.type}", ident: "{self.ident}"}}'
  
  def __eq__(self, other):
    if self.type in Var.type_map and other.type in Var.type_map:
      return Var.type_map[self.type] == Var.type_map[other.type]
    else:
      return False
  
  def __lt__(self, other):
    if self.type in Var.type_map and other.type in Var.type_map:
      return Var.type_map[self.type] < Var.type_map[other.type]
    elif self.type in Var.type_map and other.type not in Var.type_map:
      return True
    else:
      return False

class Function:
  def __init__(self, decl_arch):
    (decl, arch) = decl_arch
    self.decl = decl
    in_parens = r"\((.)+\)"
    args = re.search(in_parens, decl).group().removeprefix('(').removesuffix(')')
    decl = re.sub(in_parens, '', decl).split()
    self.return_type = decl.pop(0)
    self.intrinsic = decl.pop(0)
    self.args = [Var(arg) for arg in args.split(',')]
    self.decoded = NEONIdent(self.intrinsic)
    self.archs = arch.split('/')
    if self.args[-1].type == "const int":
      self.const = self.args[-1].ident
      self.args = self.args[:-1]
    else:
      self.const = None
    self.name = name_map[self.decoded.name] if self.decoded.name in name_map else self.decoded.name


  def __repr__(self):
    return f'Function:{{ intrinsic: "{self.intrinsic}", decoded: {self.decoded}, return_type: "{self.return_type}", args: "{self.args}" }}'
  
  def __equal__(self, other):
    var_match = [arg1 == arg2 for (arg1, arg2) in zip(self.args, other.args)]
    return functools.reduce(lambda a,b: a and b, var_match)
  
  def __lt__(self, other):
    for (arg1, arg2) in zip(self.args, other.args):
      if arg1 == arg2:
        continue
      else:
        return arg1 < arg2
    return False


In [139]:
funcs = [Function(decl_arch) for decl_arch in decl_archs]

In [140]:
v7_funcs = [f for f in funcs if "v7" in f.archs] # Filter architecture

In [141]:
vfpv3_funcs = [f for f in v7_funcs if "float16" not in f.return_type and not any(["float16" in arg.type for arg in f.args])] # remove vfpv4 stuff
vfpv3_funcs = [f for f in vfpv3_funcs if "float64" not in f.return_type and not any(["float64" in arg.type for arg in f.args])] # remove vfpv4 stuff

In [142]:
uniq_funcs = { f.decl:f for f in vfpv3_funcs}.values() # uniquify (vshll_n duped for some reason)

In [143]:
funcs = uniq_funcs

In [144]:
from pprint import pprint

sorted_names = [
  f"{name_map[func.decoded.name]} : {func.decl}" for func in sorted(funcs) if "v7" in func.archs
]

name_args = {}
for func in funcs:
  args = ', '.join([str(arg) for arg in func.args])
  key = f"{name_map[func.decoded.name]}({args})"
  if key in name_args.keys():
    name_args[key] += [func]
  else:
    name_args[key] = [func]

needs_template = {k:v for (k,v) in name_args.items() if len(v) > 1}
needs_template_sig = needs_template.keys()
needs_template_funcs = [f for v in needs_template.values() for f in v]

In [165]:

blacklist = [
  "move_high_narrow",
  "multiply_add_fused",
  "multiply_subtract_fused"
]

no_constexpr = [
  "load1",
  "load2",
  "load3",
  "load4",
  "load1_duplicate",
  "load2_duplicate",
  "load3_duplicate",
  "load4_duplicate",
  "load1_x2",
  "load1_x3",
  "load1_x4",
  "store1",
  "store2",
  "store3",
  "store4",
  "store1_x2",
  "store1_x3",
  "store1_x4",  
]

always_inline = "[[gnu::always_inline]] "

def simplify_type(type):
  parts = type.split('x')
  return parts[0] + "_v"

def generate_function(func):
  args = ', '.join([str(arg) for arg in func.args])
  arg_idents = ', '.join([arg.ident.replace('*','') for arg in func.args])
  definition = "template <> " if func in needs_template_funcs else ""
  definition += always_inline 
  definition += "nce " if func.name not in no_constexpr else "inline "
  definition += f"{func.return_type} {func.name}({args}) {{ return {func.intrinsic}({arg_idents}); }}"
  return definition

def generate_templated_function(func):
  args = ', '.join([str(arg) for arg in func.args])
  arg_idents = ', '.join([arg.ident.replace('*','') for arg in func.args])
  const_name = func.const
  args = args.replace(f', const int {const_name}', '')
  definition = f"template <int {const_name}>"
  definition += always_inline
  definition += f"nce {func.return_type} {func.name}({args}) {{ return {func.intrinsic}({arg_idents}, {const_name}); }}"
  return definition


In [166]:
#with open('arm_neon.hpp', 'w') as sys.stdout:
print('#pragma once')
print('#include <arm_neon.h>')
print('#ifdef __cplusplus')
print('''#ifdef __clang__
#define nce constexpr
#else
#define nce inline
#endif
''')
print('namespace neon {')
print('// clang-format off')
for (name_arg, func_list) in needs_template.items():
  print(f"template <typename T> nce T {name_arg};")
for func in sorted(funcs):
  if 'v7' in func.archs and func.name not in blacklist:
    if func.const != None:
      out = generate_templated_function(func)
    else:
      out = generate_function(func)
    print(out)
print('// clang-format on')
print('}  // namespace neon')
print('#endif')

#pragma once
#include <arm_neon.h>
#ifdef __cplusplus
#ifdef __clang__
#define nce constexpr
#else
#define nce inline
#endif

namespace neon {
// clang-format off
template <typename T> nce T convert(float32x2_t a);
template <typename T> nce T convert(float32x4_t a);
template <typename T> nce T convert(int32x2_t a);
template <typename T> nce T convert(int32x4_t a);
template <typename T> nce T convert(uint32x2_t a);
template <typename T> nce T convert(uint32x4_t a);
template <typename T> nce T reinterpret(int8x8_t a);
template <typename T> nce T reinterpret(int16x4_t a);
template <typename T> nce T reinterpret(int32x2_t a);
template <typename T> nce T reinterpret(float32x2_t a);
template <typename T> nce T reinterpret(uint8x8_t a);
template <typename T> nce T reinterpret(uint16x4_t a);
template <typename T> nce T reinterpret(uint32x2_t a);
template <typename T> nce T reinterpret(poly8x8_t a);
template <typename T> nce T reinterpret(poly16x4_t a);
template <typename T> nce T reinterpret(u